In [33]:
import re
import pandas as pd
import numpy as np

from fixerio import Fixerio
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from measurement.measures import Volume

In [34]:
va_df = pd.read_csv('data/VA-Price-List.csv', skiprows=1)
va_df.columns = va_df.columns.str.lower()

In [35]:
va_df = va_df[va_df.description.str.contains('WHISKEY') | va_df.description.str.contains('WHISKY')]
va_df.head()

,description,code,brand,size,age,proof,price
0,STRAIGHT BOURBON WHISKEY,21228,1792 Bourbon Full Proof,750ml,NaN,125.0,$47.99
1,STRAIGHT BOURBON WHISKEY,21232,1792 Port Finish Bourbon,750ml,NaN,88.9,$42.99
2,STRAIGHT BOURBON WHISKEY,21244,1792 Single Barrel Bourbon,750ml,NaN,98.6,$42.99
3,STRAIGHT BOURBON WHISKEY,21236,1792 Small Batch Bourbon,750ml,NaN,93.7,$33.99
4,STRAIGHT BOURBON WHISKEY,21242,1792 Sweet Wheat Bourbon,750ml,NaN,91.2,$39.99


In [36]:
va_df.describe()

,code,proof
count,871.000000,871.000000
mean,20912.814007,84.514696
std,19615.108155,13.127495
min,137.000000,0.000000
25%,6897.500000,80.000000
50%,17920.000000,80.000000
75%,26114.500000,90.000000
max,100124.000000,136.200000


In [37]:
va_df.brand.isnull().any()

False

In [38]:
def my_va_transform(s):
    """Function to transform brand
    * make lower
    * remove product types ex: 'bourbon', 'whiskey'
    """
    s = s.lower()
    
    replacements = {"^gentleman jack whiskey$": "jack daniel's gentleman jack",
                    "^pritchard": "prichard",
                    "^balcones baby blue corn whiskey$": "balcones baby blue",
                    "^canadian club rye whisky$": "canadian club",
                    "^catoctin creek roundstone rye whisky$": "catoctin creek roundstone rye",
                    "^e h taylor jr. straight rye$": "colonel e.h. taylor straight rye",
                    "^e h taylor seasoned wood$": "colonel e.h. taylor seasoned wood",
                    "^james e. pepper 1776 rye": "james e. pepper 1776 straight rye",
                    "^lock stock & barrel 16 yr straight rye whiskey$": "lock stock and barrel 16 straight rye",
                    "^michter's us 1 single barrel straight rye$": "michter's us*1",
                    "^michter's us-1 barrel strength rye": "michter's barrel strength rye",
                    "^old overholt$": "old overholt rye",
                    "^wild turkey russell's reserve rye$": "russell's reserve rye 6",
                    "\s7\s": " seven ",
                    "^defiant whisky$": "defiant",
                    "^michter's us1 sour mash$": "michter sour mash",
                    "^red stag": "jim beam red stag",
                    "^four roses 2015 limited edition small batch$": "four roses limited edition 2015",
                    "^four roses 2016 limited edition small batch$": "four roses small batch limited edition 2016",
                    "^i w harper bourbon$": "i.w. harper",
                    "^jesse james bourbon whiskey$": "original jesse james",
                    "^the": " ",
                    "scotch$": " ",
                    }
    for k, v in replacements.items():
        s = re.sub(k, v, s)
    
    replace = ['bourbon', 'craft',
               'Year Single Barrel Bourbon',
               '-year Single Barrel Bourbon',
               'whiskey', 'tennessee whiskey', 'tennessee',
               'year', 'yr', '-year single barrel',
               'year single barrel', 'label', "'s",
               'decades', 'whisky',
               ]
    
    for item in replace:
        s = s.replace(item, '')
    
    s = " ".join(s.split()) # remove extra spaces betwen words
    
    return s.strip()

In [39]:
s = 'Lock Stock & Barrel 16 Yr Straight Rye Whiskey'.lower()
replacements = {"^lock stock & barrel 16 yr straight rye whiskey$": "lock stock and barrel 16 straight rye"}
for k, v in replacements.items():
    s = re.sub(k, v, s, re.IGNORECASE)
print(s)

lock stock and barrel 16 straight rye


In [40]:
va_df['alt_brand'] = va_df['brand'].map(lambda x: my_va_transform(x))
va_df.head()

,description,code,brand,size,age,proof,price,alt_brand
0,STRAIGHT BOURBON WHISKEY,21228,1792 Bourbon Full Proof,750ml,NaN,125.0,$47.99,1792 full proof
1,STRAIGHT BOURBON WHISKEY,21232,1792 Port Finish Bourbon,750ml,NaN,88.9,$42.99,1792 port finish
2,STRAIGHT BOURBON WHISKEY,21244,1792 Single Barrel Bourbon,750ml,NaN,98.6,$42.99,1792 single barrel
3,STRAIGHT BOURBON WHISKEY,21236,1792 Small Batch Bourbon,750ml,NaN,93.7,$33.99,1792 small batch
4,STRAIGHT BOURBON WHISKEY,21242,1792 Sweet Wheat Bourbon,750ml,NaN,91.2,$39.99,1792 sweet wheat


In [41]:
def my_size_transform(s):
    if 'ml' in s:
        v = Volume(milliliter=s.replace('ml', ''))
    elif 'L' in s:
        v = Volume(liter=s.replace('L', ''))
    return v.us_oz

In [42]:
va_df['oz'] = va_df['size'].map(lambda x: my_size_transform(x))
va_df['alt_price'] = va_df['price'].str.replace('$', '')
va_df['alt_age'] = va_df['age'].str.replace('YR', '')

In [43]:
wa_df = pd.read_csv('data/Meta-Critic Whisky Database.csv')
wa_df.columns = wa_df.columns.str.lower().str.replace(' ', '_')

In [44]:
wa_df.head()

,whisky,meta_critic,stdev,#,cost,class,super_cluster,cluster,country,type
0,Glenfarclas 40yo,9.25,0.30,11,$$$$$+,SingleMalt-like,ABC,A,Scotland,Malt
1,Amrut Greedy Angels (8yo and 10yo),9.20,0.20,6,$$$$$+,SingleMalt-like,ABC,C,India,Malt
2,Redbreast 21yo,9.19,0.32,13,$$$$$,SingleMalt-like,ABC,C,Ireland,Malt
3,Amrut Spectrum,9.18,0.25,8,$$$$$,SingleMalt-like,ABC,C,India,Malt
4,Highland Park 25yo,9.17,0.24,13,$$$$$+,SingleMalt-like,ABC,C,Scotland,Malt


In [45]:
w_archive = pd.read_csv('data/Reddit Whisky Network Review Archive - Review Archive.csv',
                        names=['timestamp', 'whisky_name', 'reviewer_username', 'link',
                               'rating', 'style', 'bottle_price', 'review_date'],
                        skiprows=1,
                        parse_dates=['timestamp', 'review_date'])

In [46]:
w_archive['rating'] = pd.to_numeric(w_archive['rating'], errors='coerce')
#w_archive['bottle_price'] = pd.to_numeric(w_archive['bottle_price'], errors='coerce')
w_archive['timestamp'] = pd.to_datetime(w_archive['timestamp'], errors='coerce')
w_archive['review_date'] = pd.to_datetime(w_archive['review_date'], errors='coerce')
w_archive['whisky_name'] = w_archive.whisky_name.str.lower()
w_archive['style'] = w_archive['style'].str.lower()
w_archive.dtypes

timestamp            datetime64[ns]
whisky_name                  object
reviewer_username            object
link                         object
rating                      float64
style                        object
bottle_price                 object
review_date          datetime64[ns]
dtype: object

In [47]:
w_archive.head(20)

,timestamp,whisky_name,reviewer_username,link,rating,style,bottle_price,review_date
0,2012-12-14 10:03:18,100 pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68.0,blend,NaN,2012-12-14
1,2015-06-22 11:40:00,11 wells minnesota 13 white whiskey,KozureOkami,http://www.reddit.com/r/worldwhisky/comments/3...,75.0,white,30,2015-06-22
2,2016-10-31 16:14:05,1792 full proof,dmsn7d,https://www.reddit.com/r/bourbon/comments/5aez...,85.0,bourbon,NaN,2016-10-31
3,2016-10-19 11:20:32,1792 full proof,mentel42,https://www.reddit.com/r/bourbon/comments/56f1...,87.0,bourbon,50,2016-10-08
4,NaT,1792 full proof,signde,https://www.reddit.com/r/bourbon/comments/52b8...,80.0,bourbon,45,2016-09-11
5,2017-02-27 21:07:42,1792 high rye,spg2469,https://www.reddit.com/r/bourbon/comments/5wli...,86.0,bourbon,36,2017-02-27
6,2015-12-04 10:13:34,1792 port finish,kdz13,https://www.reddit.com/r/bourbon/comments/3v2y...,79.0,bourbon,NaN,2015-12-02
7,2016-01-15 16:26:11,1792 port finish,lvl1dramacenter,https://www.reddit.com/r/bourbon/comments/410u...,83.0,bourbon,38,2016-01-14
8,2015-11-13 12:52:10,1792 port finish,MicMumbles,https://www.reddit.com/r/bourbon/comments/3ryw...,76.0,bourbon,42,2015-11-08
9,2016-03-13 11:53:39,1792 port finish,PalmMoon,https://www.reddit.com/r/bourbon/comments/4a6x...,75.0,bourbon,49,2016-03-12


In [142]:
fxrio = Fixerio(base='USD')
conversion = fxrio.latest()
import math

def isnumber(num):
    return all(char.isdigit() for char in num)

def transform_currency(s):
    if 'CAD' in str(s) or 'CDN' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['CAD'])
        return new_price
    elif '£' in str(s) or 'GBP' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['GBP'])
        return new_price
    elif 'AUD' in str(s) or 'AUS' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['AUD'])
        return new_price
    elif 'NZD' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['NZD'])
        return new_price
    elif 'EUR' in str(s) or '€' in str(s) or 'Euro' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['EUR'])
        return new_price
    elif 'SEK' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['SEK'])
        return new_price
    elif 'RMB' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['CNY'])
        return new_price
    elif 'DKR' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['DKK'])
        return new_price
    elif 'HKD' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['HKD'])
        return new_price
    elif 'JPY' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['JPY'])
        return new_price
    elif 'ZAR' in str(s):
        price = re.sub(r"""[^0-9]+""", '', s)
        new_price = float(price) / float(conversion['rates']['ZAR'])
        return new_price
    elif type(s) == str and isnumber(s):
        # USD
        return s
    elif type(s) == float and math.isnan(s):
        # NAN
        return s
    elif re.match(r"""^\$*\d+\.*\d+$""", s):
        s = s.replace('$', '')
        return s
    else:
        # Convert everything else to NAN
        print(s)
        return None

In [143]:
w_archive['alt_brand'] = w_archive['whisky_name'].map(lambda x: my_va_transform(x))
w_archive['alt_bottle_price'] = w_archive['bottle_price'].map(lambda x: transform_currency(x))
w_archive.head(100)

$270 ARS


,timestamp,whisky_name,reviewer_username,link,rating,style,bottle_price,review_date,alt_brand,alt_bottle_price
0,2012-12-14 10:03:18,100 pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68.0,blend,NaN,2012-12-14,100 pipers,NaN
1,2015-06-22 11:40:00,11 wells minnesota 13 white whiskey,KozureOkami,http://www.reddit.com/r/worldwhisky/comments/3...,75.0,white,30,2015-06-22,11 wells minnesota 13 white,30
2,2016-10-31 16:14:05,1792 full proof,dmsn7d,https://www.reddit.com/r/bourbon/comments/5aez...,85.0,bourbon,NaN,2016-10-31,1792 full proof,NaN
3,2016-10-19 11:20:32,1792 full proof,mentel42,https://www.reddit.com/r/bourbon/comments/56f1...,87.0,bourbon,50,2016-10-08,1792 full proof,50
4,NaT,1792 full proof,signde,https://www.reddit.com/r/bourbon/comments/52b8...,80.0,bourbon,45,2016-09-11,1792 full proof,45
5,2017-02-27 21:07:42,1792 high rye,spg2469,https://www.reddit.com/r/bourbon/comments/5wli...,86.0,bourbon,36,2017-02-27,1792 high rye,36
6,2015-12-04 10:13:34,1792 port finish,kdz13,https://www.reddit.com/r/bourbon/comments/3v2y...,79.0,bourbon,NaN,2015-12-02,1792 port finish,NaN
7,2016-01-15 16:26:11,1792 port finish,lvl1dramacenter,https://www.reddit.com/r/bourbon/comments/410u...,83.0,bourbon,38,2016-01-14,1792 port finish,38
8,2015-11-13 12:52:10,1792 port finish,MicMumbles,https://www.reddit.com/r/bourbon/comments/3ryw...,76.0,bourbon,42,2015-11-08,1792 port finish,42
9,2016-03-13 11:53:39,1792 port finish,PalmMoon,https://www.reddit.com/r/bourbon/comments/4a6x...,75.0,bourbon,49,2016-03-12,1792 port finish,49


In [136]:
va_new_brand = va_df['alt_brand'].tolist()
wa_new_whisky = w_archive['alt_brand'].tolist()

In [137]:
exact_matches = set()
for brand in va_new_brand:
    if brand in wa_new_whisky:
        exact_matches.add(brand)
print('Matches: {}'.format(len(exact_matches)))
exact_matches = list(exact_matches)
exact_matches.sort()
print('\n'.join(exact_matches))

Matches: 210
1792 full proof
1792 high rye
1792 port finish
1792 small batch
1792 sweet wheat
ancient age
angel envy rye
ardbeg auriverdes
ardbeg corryvreckan
ardbeg dark cove committee release
ardbeg perpetuum
auchentoshan three wood
balcones baby blue
balvenie 14 caribbean cask
basil hayden
belle meade
black grouse
black velvet
booker
booker rye
breckenridge
buffalo trace
bulleit
bulleit 10
bulleit rye
bunnahabhain toiteach
canadian club
canadian mist
caol ila 12
caol ila 30
catoctin creek roundstone rye
chivas regal 12
chivas regal 18
chivas regal extra
clan macgregor
copper fox rye
crown royal
crown royal black
crown royal hand selected barrel
crown royal monarch 75th anniversary
crown royal northern harvest rye
crown royal special reserve
cutty sark
dalwhinnie distiller edition
david nicholson 1843
defiant
dewar highlander honey
dewar white
eagle rare 10
early times
elijah craig 12
elijah craig small batch
evan williams 1783
evan williams black
evan williams green
evan williams si

In [30]:
for brand in va_new_brand:
    matches = process.extract(brand, wa_new_whisky)
    if matches[0][1] != 100:
        print(brand, matches[0])

1792 single barrel ('abraham bowman double barrel', 86)
abraham bowman gingerbread cocoa finish ('abraham bowman gingerbread beer finish', 88)
abraham bowman wheat limited edition ('1792 sweet wheat', 86)
ancient age traveler ('ancient age', 90)
ancient ancient age ('ancient age 100 bib', 95)
ancient ancient age ('ancient age 100 bib', 95)
angel envy port barrel ('angel envy', 90)
baker ('baker 107', 90)
barterhouse ('orphan barrel barterhouse 20', 90)
belle meade sherry finish ('belle meade 9 sherry finished', 93)
belle meade single barrel ('belle meade', 90)
benchmark no. 8 ('benchmark', 90)
benchmark no. 8 ('benchmark', 90)
bib & tucker ('bib & tucker 6 small batch', 90)
bird dog 100ml variety pack #2 ('arran smugglers\' series vol. 2 "the high seas"', 86)
bird dog ('buffalo trace white dog #1', 86)
bird dog small batch ('aberlour that boutique-y company batch 2', 86)
blanton single barrel ('blanton original single barrel', 95)
blanton single barrel ('blanton original single barrel'

In [545]:
#va_df[va_df['new_brand'].str.contains('james')]

In [546]:
#w_archive[w_archive['new_whisky_name'].str.contains('stag')]

In [144]:
va_df.to_csv('data/alt_va_prices.csv', index=False)

In [145]:
w_archive.to_csv('data/alt_reddit_archive.csv')